## Vignetting Demo

This notebook demonstrates the adjustment of limiting magnitude at the object position to account for vignetting.

In [ ]:
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
from sorcha.modules.PPVignetting import vignettingEffects

We begin by loading in a set of randomised artificial observations which were all generated to lie on the same field, within 2.1 degrees of field centre. To begin with, these observations all have the same five sigma depth at source, or limiting magnitude.

In [ ]:
def get_sql_data(database, rows_start, nrows):
    
    con = sql.connect(database)
    observations = pd.read_sql("""SELECT observationId, observationStartMJD as observationStartMJD_TAI, visitTime, visitExposureTime, filter, seeingFwhmGeom as seeingFwhmGeom_arcsec, seeingFwhmEff as seeingFwhmEff_arcsec, fiveSigmaDepth as fieldFiveSigmaDepth_mag, fieldRA as fieldRA_deg, fieldDec as fieldDec_deg, rotSkyPos as fieldRotSkyPos_deg FROM observations order by observationId LIMIT """ + str(rows_start) + ',' + str(nrows), con)
    
    return observations

In [ ]:
db_path = "oneline_v2.0.db"
LSSTdf = get_sql_data(db_path, 0,1)

In [ ]:
dfobs = pd.read_csv("footprintFilterValidationObservations.csv", sep='\s+')

In [ ]:
dfobs = pd.merge(dfobs, LSSTdf, left_on="FieldID", right_on="observationId", how="left")

In [ ]:
dfobs

In [ ]:
dfobs['fieldFiveSigmaDepth_mag']

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))
sc=ax.scatter(dfobs['fieldRA_deg']-dfobs['RA_deg'],dfobs['fieldDec_deg']-dfobs['Dec_deg'], s=0.1, c=dfobs['fieldFiveSigmaDepth_mag'].values)
ax.set_ylabel('distance from field centre RA (deg)')
ax.set_xlabel('distance from field centre Dec (deg)')
ax.set_aspect('equal', adjustable='box')
fig.colorbar(sc, ax=ax)

In the above plot, the points have been shaded according to their limiting magnitude.

Now we apply the effects of vignetting and replot. 

In [ ]:
dfobs['fiveSigmaDepth_mag'] = vignettingEffects(dfobs)

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))
sc=ax.scatter(dfobs['fieldRA_deg']-dfobs['RA_deg'],dfobs['fieldDec_deg']-dfobs['Dec_deg'], s=0.1, c=dfobs['fiveSigmaDepth_mag'].values)
ax.set_ylabel('distance from field centre RA (deg)')
ax.set_xlabel('distance from field centre Dec (deg)')
fig.colorbar(sc, ax=ax)
ax.set_aspect('equal', adjustable='box')
plt.show()

The effects of vignetting can thus be seen.